In [6]:
# =============================================================================
# Конфигурация проекта: загрузка из файла или создание нового
# =============================================================================
import os
import yaml
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
from PIL import Image
import glob

CONFIG_FILE = "config.yaml"

DEFAULT_CONFIG = {
    "debug_mode": False,
    "debug_train_size": 500,
    "debug_val_size": 100,
    "data_dir": "ogyeiv2",
    "batch_size": 32,
    "epochs": 10,
    "learning_rate": 1e-3,
    "img_size": 224,
    "inference_dir": "vdv-imgclass",
    "model_save_path": "meds_classifier.pt",
    "inference_output_csv": "vdv-imgclass.csv",
    "target_val_acc": 75.0,
    "dataset_structure": "split"  # "split" или "flat"
}

def load_or_create_config(config_file=CONFIG_FILE):
    if os.path.exists(config_file):
        with open(config_file, 'r', encoding='utf-8') as f:
            config = yaml.safe_load(f)
        print(f"✅ Конфиг загружен из {config_file}")
    else:
        config = DEFAULT_CONFIG.copy()
        with open(config_file, 'w', encoding='utf-8') as f:
            yaml.dump(config, f, default_flow_style=False, allow_unicode=True)
        print(f"🆕 Конфиг создан: {config_file}")
    return config

CONFIG = load_or_create_config()
CLASS_NAMES = None

🆕 Конфиг создан: config.yaml


In [6]:
# =============================================================================
# Этап 1. Загрузка и предобработка данных.
# Поддержка двух структур: "split" (train/test) и "flat" (единый набор)
# =============================================================================

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((CONFIG["img_size"], CONFIG["img_size"])),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((CONFIG["img_size"], CONFIG["img_size"])),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

if CONFIG["dataset_structure"] == "split":
    print("📁 Используется разделённая структура (train/test)")
    train_dataset_raw = datasets.ImageFolder(root=os.path.join(CONFIG["data_dir"], "train"))
    val_dataset_raw = datasets.ImageFolder(root=os.path.join(CONFIG["data_dir"], "test"))
    assert train_dataset_raw.classes == val_dataset_raw.classes, "Классы в train и test не совпадают!"
    CLASS_NAMES = train_dataset_raw.classes
    CONFIG["num_classes"] = len(CLASS_NAMES)

    train_dataset = datasets.ImageFolder(
        root=os.path.join(CONFIG["data_dir"], "train"),
        transform=data_transforms['train']
    )
    val_dataset = datasets.ImageFolder(
        root=os.path.join(CONFIG["data_dir"], "test"),
        transform=data_transforms['val']
    )

elif CONFIG["dataset_structure"] == "flat":
    print("📁 Используется плоская структура (единый набор классов)")
    full_dataset_original = datasets.ImageFolder(root=CONFIG["data_dir"])
    CLASS_NAMES = full_dataset_original.classes
    CONFIG["num_classes"] = len(CLASS_NAMES)

    # Создаём датасет с train-трансформами
    full_dataset_train = datasets.ImageFolder(root=CONFIG["data_dir"], transform=data_transforms['train'])
    # Разделение индексов
    val_size = int(0.2 * len(full_dataset_train))
    train_size = len(full_dataset_train) - val_size
    train_indices, val_indices = random_split(full_dataset_train, [train_size, val_size], generator=torch.Generator().manual_seed(42))

    train_dataset = Subset(full_dataset_train, train_indices.indices)
    # Валидационный датасет — с val-трансформами
    full_dataset_val = datasets.ImageFolder(root=CONFIG["data_dir"], transform=data_transforms['val'])
    val_dataset = Subset(full_dataset_val, val_indices.indices)

else:
    raise ValueError(f"Неизвестная структура датасета: {CONFIG['dataset_structure']}. Используйте 'split' или 'flat'.")

# Дебаг-режим
if CONFIG["debug_mode"]:
    train_dataset = Subset(train_dataset, range(min(CONFIG["debug_train_size"], len(train_dataset))))
    val_dataset = Subset(val_dataset, range(min(CONFIG["debug_val_size"], len(val_dataset))))

train_loader = DataLoader(train_dataset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=2)

print(f"✅ Количество классов: {CONFIG['num_classes']}")
print(f"✅ Train: {len(train_dataset)} изображений")
print(f"✅ Val: {len(val_dataset)} изображений")
print(f"✅ Примеры классов: {CLASS_NAMES[:5]}")

📁 Используется разделённая структура (train/test)
✅ Количество классов: 84
✅ Train: 2352 изображений
✅ Val: 504 изображений
✅ Примеры классов: ['acc_long_600_mg', 'advil_ultra_forte', 'akineton_2_mg', 'algoflex_forte_dolo_400_mg', 'algoflex_rapid_400_mg']


In [7]:
# =============================================================================
# Этап 2. Объявление модели
# =============================================================================

model = torchvision.models.mobilenet_v3_small(pretrained=True)

# Заморозка признаковых слоёв
for param in model.features.parameters():
    param.requires_grad = False

# Замена головы
model.classifier[3] = nn.Linear(model.classifier[3].in_features, CONFIG["num_classes"])

model

/home/ubuntu/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.83M/9.83M [00:00<00:00, 20.9MB/s]


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [8]:
# =============================================================================
# Этап 3. Обучение с прогресс-баром (tqdm), временем и метриками
# Модель сохраняется в CONFIG["model_save_path"], только если Val Acc улучшилась
# =============================================================================

import time
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Устройство: {device}")

model = torchvision.models.mobilenet_v3_small(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False
model.classifier[3] = nn.Linear(model.classifier[3].in_features, CONFIG["num_classes"])

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=CONFIG["learning_rate"])

best_val_acc = 0.0
target_reached = False

for epoch in range(CONFIG["epochs"]):
    print(f"\n{'='*60}")
    print(f"Эпоха {epoch+1}/{CONFIG['epochs']}")
    print(f"{'='*60}")

    # ===== Обучение =====
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Используем tqdm для красивого прогресс-бара
    pbar = tqdm(train_loader, desc="Обучение", unit="batch", leave=True)
    
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,(labels))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Текущие метрики
        avg_loss = running_loss / (pbar.n + 1)
        train_acc = 100 * correct_train / total_train

        # Обновляем описание прогресс-бара
        pbar.set_postfix({
            "Loss": f"{avg_loss:.4f}",
            "Acc": f"{train_acc:.2f}%"
        })

    train_acc_final = 100 * correct_train / total_train

    # ===== Валидация =====
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0
    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Валидация", unit="batch", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_acc = 100 * correct_val / total_val
    avg_val_loss = val_loss / len(val_loader)

    print(f"\n📌 Итоги эпохи {epoch+1}:")
    print(f"  Train Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc_final:.2f}%")
    print(f"  Val Loss: {avg_val_loss:.4f}   | Val Acc: {val_acc:.2f}%")

    # ===== Ранняя остановка =====
    if val_acc >= CONFIG["target_val_acc"]:
        print(f"🎯 Достигнут целевой порог точности ({CONFIG['target_val_acc']}%) — обучение остановлено.")
        target_reached = True

    # ===== Сохранение лучшей модели =====
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        model.cpu()
        torch.save(model.state_dict(), CONFIG["model_save_path"])
        print(f"✅ Модель сохранена (лучшая Val Acc: {val_acc:.2f}%) → {CONFIG['model_save_path']}")
        model.to(device)

    if target_reached:
        break

Устройство: cuda

Эпоха 1/10


Обучение: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [01:37<00:00,  1.32s/batch, Loss=3.8305, Acc=11.73%]
                                                                                                                                                                                     


📌 Итоги эпохи 1:
  Train Loss: 3.8305 | Train Acc: 11.73%
  Val Loss: 4.0701   | Val Acc: 2.78%
✅ Модель сохранена (лучшая Val Acc: 2.78%) → meds_classifier.pt

Эпоха 2/10


Обучение: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [01:38<00:00,  1.33s/batch, Loss=2.5974, Acc=33.29%]
                                                                                                                                                                                     


📌 Итоги эпохи 2:
  Train Loss: 2.5974 | Train Acc: 33.29%
  Val Loss: 3.7386   | Val Acc: 6.35%
✅ Модель сохранена (лучшая Val Acc: 6.35%) → meds_classifier.pt

Эпоха 3/10


...


📌 Итоги эпохи 8:
  Train Loss: 1.2127 | Train Acc: 63.78%
  Val Loss: 1.0654   | Val Acc: 66.67%

Эпоха 9/10


Обучение: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [01:38<00:00,  1.33s/batch, Loss=1.1625, Acc=65.52%]
                                                                                                                                                                                     


📌 Итоги эпохи 9:
  Train Loss: 1.1625 | Train Acc: 65.52%
  Val Loss: 0.9491   | Val Acc: 73.81%
✅ Модель сохранена (лучшая Val Acc: 73.81%) → meds_classifier.pt

Эпоха 10/10


Обучение: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [01:38<00:00,  1.33s/batch, Loss=1.1119, Acc=66.67%]
                                                                                                                                                                                     


📌 Итоги эпохи 10:
  Train Loss: 1.1119 | Train Acc: 66.67%
  Val Loss: 0.9540   | Val Acc: 73.81%


In [11]:
# =============================================================================
# Этап 4. Оценка качества (работает всегда — на CPU)
# =============================================================================

import os
import yaml
from torchvision import models
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

# Загрузка конфига (на случай, если запускаете отдельно)
with open("config.yaml", 'r', encoding='utf-8') as f:
    CONFIG = yaml.safe_load(f)

# Получение CLASS_NAMES (аналогично Этапу 5)
from torchvision.datasets import ImageFolder
if CONFIG["dataset_structure"] == "split":
    CLASS_NAMES = ImageFolder(root=os.path.join(CONFIG["data_dir"], "train")).classes
else:
    CLASS_NAMES = ImageFolder(root=CONFIG["data_dir"]).classes

# Пересоздание модели на CPU
model = models.mobilenet_v3_small(pretrained=False)
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, len(CLASS_NAMES))
model.load_state_dict(torch.load(CONFIG["model_save_path"], map_location='cpu'))
model.eval()  # важно!

# Загрузка валидационного датасета (аналогично Этапу 1, но только val)
from torchvision import transforms
val_transform = transforms.Compose([
    transforms.Resize((CONFIG["img_size"], CONFIG["img_size"])),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

if CONFIG["dataset_structure"] == "split":
    val_dataset = ImageFolder(root=os.path.join(CONFIG["data_dir"], "test"), transform=val_transform)
else:
    # Для flat-структуры нужно повторно разделить, но проще использовать val_loader из train-разделения
    # Однако для точности оценки — лучше использовать оригинальный test, если он есть
    # В данном случае, если flat — мы не можем точно воссоздать val без сохранения индексов
    # Поэтому предположим, что в flat режиме test нет, и вы использовали random_split
    # Но для корректной оценки качества — рекомендуется использовать split-структуру
    raise NotImplementedError("Оценка качества для flat-структуры требует сохранения val-индексов. Используйте dataset_structure: split.")

val_loader = DataLoader(val_dataset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=2)

# Оценка
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in val_loader:
        # inputs и labels уже на CPU (ImageFolder + DataLoader без .to())
        outputs = model(inputs)  # модель на CPU → всё ок
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.numpy())
        all_preds.extend(predicted.numpy())

# Далее — как раньше
accuracy = accuracy_score(all_labels, all_preds)
print(f"Общая accuracy: {accuracy * 100:.2f}%")

cm = confusion_matrix(all_labels, all_preds)
error_counts = cm.sum(axis=1) - np.diag(cm)
class_error_pairs = list(zip(CLASS_NAMES, error_counts))
sorted_by_errors = sorted(class_error_pairs, key=lambda x: x[1], reverse=True)

print("\nТоп-5 классов с наибольшим количеством ошибок:")
for cls, err in sorted_by_errors[:5]:
    print(f" - {cls}: {int(err)} ошибок")

print("\nТоп-5 классов с наименьшим количеством ошибок:")
for cls, err in sorted_by_errors[-5:]:
    print(f" - {cls}: {int(err)} ошибок")

print("\nКлассы без ошибок:")
zero_error = [cls for cls, err in class_error_pairs if err == 0]
if zero_error:
    for cls in zero_error:
        print(f" - {cls}")
else:
    print(" - Таких классов нет")

/home/ubuntu/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Общая accuracy: 73.81%

Топ-5 классов с наибольшим количеством ошибок:
 - atorvastatin_teva_20_mg: 6 ошибок
 - jutavit_cink: 6 ошибок
 - concor_10_mg: 5 ошибок
 - quamatel_40_mg: 5 ошибок
 - theospirex_150_mg: 5 ошибок

Топ-5 классов с наименьшим количеством ошибок:
 - strepsils: 0 ошибок
 - urzinol: 0 ошибок
 - vita_c: 0 ошибок
 - voltaren_dolo_rapid_25_mg: 0 ошибок
 - zadex_60_mg: 0 ошибок

Классы без ошибок:
 - advil_ultra_forte
 - akineton_2_mg
 - algoflex_rapid_400_mg
 - algopyrin_500_mg
 - bila_git
 - calci_kid
 - cataflam_dolo_25_mg
 - concor_5_mg
 - diclopram_75-mg_20-mg
 - dorithricin_mentol
 - dulsevia_60_mg
 - laresin_10_mg
 - metothyrin_10_mg
 - mezym_forte_10_000_egyseg
 - milgamma
 - naprosyn_250_mg
 - novo_c_plus
 - ocutein
 - salazopyrin_en_500_mg
 - strepfen_8_75_mg
 - strepsils
 - urzinol
 - vita_c
 - voltaren_dolo_rapid_25_mg
 - zadex_60_mg


In [5]:
# =============================================================================
# Этап 5. Инференс (автономный — работает без датасета!)
# Требует только: meds_classifier.pt + classes.txt + config.yaml
# =============================================================================

import os
import yaml
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import glob
import pandas as pd

# --- Загрузка конфигурации ---
with open("config.yaml", 'r', encoding='utf-8') as f:
    CONFIG = yaml.safe_load(f)

# --- Загрузка списка классов из classes.txt ---
try:
    with open("classes.txt", "r", encoding="utf-8") as f:
        CLASS_NAMES = [line.strip() for line in f if line.strip()]
    print(f"✅ Загружено {len(CLASS_NAMES)} классов из classes.txt")
except FileNotFoundError:
    raise FileNotFoundError(
        "Файл classes.txt не найден. "
        "Создайте его в корне проекта и запишите в него названия классов по одному на строку."
    )

# --- Подготовка инференс-директории ---
os.makedirs(CONFIG["inference_dir"], exist_ok=True)

# --- Трансформы ---
inference_transform = transforms.Compose([
    transforms.Resize((CONFIG["img_size"], CONFIG["img_size"])),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- Поиск изображений ---
image_paths = (
    glob.glob(os.path.join(CONFIG["inference_dir"], "*.jpg")) +
    glob.glob(os.path.join(CONFIG["inference_dir"], "*.jpeg")) +
    glob.glob(os.path.join(CONFIG["inference_dir"], "*.png"))
)

if not image_paths:
    print(f"⚠️ Нет изображений в {CONFIG['inference_dir']}")
else:
    # --- Загрузка модели ---
    model = torchvision.models.mobilenet_v3_small(pretrained=False)
    model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, len(CLASS_NAMES))
    model.load_state_dict(torch.load(CONFIG["model_save_path"], map_location='cpu'))
    model.eval()

    results = []
    with torch.no_grad():
        for img_path in image_paths:
            try:
                img = Image.open(img_path).convert("RGB")
                inp = inference_transform(img).unsqueeze(0)
                out = model(inp)
                prob = torch.softmax(out, dim=1)
                conf, idx = torch.max(prob, 1)
                cls_name = CLASS_NAMES[idx.item()]
                results.append({
                    "filename": os.path.basename(img_path),
                    "class": cls_name,
                    "confidence": conf.item()
                })
                print(f"{os.path.basename(img_path)} - {cls_name} - {conf.item():.4f}")
            except Exception as e:
                print(f"Ошибка при обработке {img_path}: {e}")

    if results:
        pd.DataFrame(results).to_csv(CONFIG["inference_output_csv"], index=False)
        print(f"\n✅ Результаты сохранены в {CONFIG['inference_output_csv']}")

✅ Загружено 84 классов из classes.txt
advil_ultra_forte_s_037.jpg - advil_ultra_forte - 0.9642
akineton_2_mg_u_013.jpg - algopyrin_500_mg - 0.8987
apranax_550_mg_u_010.jpg - apranax_550_mg - 0.4894
aspirin_ultra_500_mg_u_014.jpg - aspirin_ultra_500_mg - 0.8088
atoris_20_mg_s_039.jpg - atoris_20_mg - 0.4635
betaloc_50_mg_u_005.jpg - betaloc_50_mg - 0.9603
calci_kid_u_014.jpg - calci_kid - 0.9997
cataflam_dolo_25_mg_s_024.jpg - cataflam_dolo_25_mg - 0.9926
cataflam_dolo_25_mg_u_012.jpg - cataflam_dolo_25_mg - 0.9154
strepsils_u_001.jpg - strepsils - 0.9748

✅ Результаты сохранены в vdv-imgclass.csv


Readme доступно на репозитории по ссылке: https://github.com/vdv-777/VDV-imgclass-mobilenet

## 📊 Итоги обучения на полном датасете (10 эпох, CUDA)

- **Общая точность (accuracy): `73.81%`** — модель демонстрирует работоспособный, хотя и не максимальный результат.  
- **Сложные для классификации таблетки**:  
  Наибольшее число ошибок допущено для `atorvastatin_teva_20_mg` и `jutavit_cink` (по 6 ошибок), а также `concor_10_mg`, `quamatel_40_mg`, `theospirex_150_mg` (по 5 ошибок). Вероятно, эти препараты визуально схожи с другими классами или представлены в неоднозначных ракурсах/освещении.  
- **Надёжно распознаваемые таблетки**:  
  **25 классов** были классифицированы **без единой ошибки**, включая `strepsils`, `urzinol`, `vita_c`, `voltaren_dolo_rapid_25_mg`, `zadex_60_mg` и другие. Это свидетельствует о высокой устойчивости модели к большинству образцов.  
- **Рекомендации**:  
  Для повышения общей accuracy целесообразно провести анализ изображений «проблемных» классов — возможно, требуется аугментация, балансировка датасета или дообучение с фокусом на наиболее сложных примерах.